<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/tasty_annual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import os, time
import numpy as np
import math
import time
import json

from pathlib import Path

import pandas as pd

In [86]:
file_path = '/content/drive/MyDrive/tastyworks_transactions_x5706_2021-01-01_2021-12-31.csv'
file_exists = os.path.exists(file_path)
if file_exists is True:
  tasty_df = pd.read_csv(file_path)
  #tasty_df = tasty_df.reset_index()  # make sure indexes pair with number of rows
  if 'Status' not in tasty_df:
    tasty_df['Status'] = ''
  print("Tasty History loaded:", file_path)
else:
  print("Tasty History does not exists:", file_path)


Tasty History loaded: /content/drive/MyDrive/tastyworks_transactions_x5706_2021-01-01_2021-12-31.csv


In [87]:
nan = np.nan
def find_closed_transactions(symbol, instrumentType, df):
  return df.query('(Action.str.contains("CLOSE") or Action=="-") and Symbol=="' + str(symbol) + '" and `Instrument Type`=="' + str(instrumentType) + '"', engine='python') 

def find_open_transactions(symbol, instrumentType, df):
  return df.query('Action.str.contains("OPEN")  and Symbol=="' + str(symbol) + '" and `Instrument Type`=="' + str(instrumentType) + '"', engine='python') 

In [88]:
for index, row in tasty_df.iterrows():
  tasty_df.loc[index,'Status'] = ''
  if len(str(row['Action'])) < 4:
    tasty_df.loc[index,'Action'] = "-"
  if "TRANSFER FROM INTERACTIVE" in str(row['Description']):
    tasty_df.loc[index,'Status'] = "OPEN"
    # print(index, row['Date'], row['Action'], row['Symbol'], row['Quantity'])


In [89]:
for index, row in tasty_df.iterrows():
  closed_df = find_closed_transactions(row['Symbol'],row['Instrument Type'], tasty_df)
  open_df = find_open_transactions(row['Symbol'],row['Instrument Type'], tasty_df)

  closed_quantity = np.sum(closed_df['Quantity'])
  open_quantity = np.sum(open_df['Quantity'])
  
  status = ''
  if closed_quantity > 0:
    if open_quantity == closed_quantity:
      status = 'CLOSED'
    else:
      status = 'PARTIALLY CLOSED'
  elif open_quantity > 0:
    status = 'OPEN'
  
  for closed_index, closed_row in closed_df.iterrows():
    tasty_df.loc[closed_index,'Status'] = status
  for open_index, open_row in open_df.iterrows():
    tasty_df.loc[open_index,'Status'] = status
  
  print(index, row['Date'], row['Action'], row['Symbol'], row['Quantity'], closed_quantity,  tasty_df.loc[index,'Status'])

tasty_df.to_csv(file_path)

0 2021-12-31T23:00:00+0100 - QQQ 0.0 0.0 OPEN
1 2021-12-31T23:00:00+0100 - QQQ 0.0 0.0 OPEN
2 2021-12-31T23:00:00+0100 - QQQ 0.0 0.0 OPEN
3 2021-12-31T23:00:00+0100 - QQQ 0.0 0.0 OPEN
4 2021-12-31T23:00:00+0100 - ARKG 0.0 100.0 CLOSED
5 2021-12-31T23:00:00+0100 - ARKG 0.0 100.0 CLOSED
6 2021-12-31T23:00:00+0100 - nan 0.0 0.0 
7 2021-12-30T23:00:00+0100 - VTI 0.0 0.0 OPEN
8 2021-12-30T23:00:00+0100 - VTI 0.0 0.0 OPEN
9 2021-12-30T20:53:07+0100 BUY_TO_OPEN AFRM  220218P00100000 1.0 0.0 OPEN
10 2021-12-30T20:53:07+0100 SELL_TO_OPEN AFRM  220218P00105000 1.0 0.0 OPEN
11 2021-12-30T20:07:05+0100 SELL_TO_OPEN AMZN  220218P03120000 1.0 0.0 OPEN
12 2021-12-30T20:07:05+0100 BUY_TO_OPEN AMZN  220218P03100000 1.0 0.0 OPEN
13 2021-12-30T19:13:32+0100 SELL_TO_OPEN COST  220121C00570000 1.0 0.0 OPEN
14 2021-12-30T19:13:32+0100 BUY_TO_OPEN COST  220218C00570000 1.0 0.0 OPEN
15 2021-12-30T18:32:00+0100 BUY_TO_CLOSE BNGO  220520C00007000 1.0 1.0 CLOSED
16 2021-12-30T18:09:04+0100 SELL_TO_CLOSE ZM 3.0 4